In [21]:
import pandas as pd
import numpy as np
import os
import json
from shapely.geometry import Polygon
import cv2
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon as Polygon_patch
from matplotlib.patches import Rectangle as Rectangle_patch

In [22]:
yolo_part_mapping = {0: 'Traffic-sings',
 1: 'forb_ahead',
 2: 'forb_left',
 3: 'forb_overtake',
 4: 'forb_right',
 5: 'forb_speed_over_10',
 6: 'forb_speed_over_100',
 7: 'forb_speed_over_130',
 8: 'forb_speed_over_20',
 9: 'forb_speed_over_30',
 10: 'forb_speed_over_40',
 11: 'forb_speed_over_5',
 12: 'forb_speed_over_50',
 13: 'forb_speed_over_60',
 14: 'forb_speed_over_70',
 15: 'forb_speed_over_80',
 16: 'forb_speed_over_90',
 17: 'forb_stopping',
 18: 'forb_trucks',
 19: 'forb_u_turn',
 20: 'forb_weight_over_3.5t',
 21: 'forb_weight_over_7.5t',
 22: 'info_bus_station',
 23: 'info_crosswalk',
 24: 'info_highway',
 25: 'info_one_way_traffic',
 26: 'info_parking',
 27: 'info_taxi_parking',
 28: 'mand_bike_lane',
 29: 'mand_left',
 30: 'mand_left_right',
 31: 'mand_pass_left',
 32: 'mand_pass_left_right',
 33: 'mand_pass_right',
 34: 'mand_right',
 35: 'mand_roundabout',
 36: 'mand_straigh_left',
 37: 'mand_straight',
 38: 'mand_straight_right',
 39: 'prio_give_way',
 40: 'prio_priority_road',
 41: 'prio_stop',
 42: 'warn_children',
 43: 'warn_construction',
 44: 'warn_crosswalk',
 45: 'warn_cyclists',
 46: 'warn_domestic_animals',
 47: 'warn_other_dangers',
 48: 'warn_poor_road_surface',
 49: 'warn_roundabout',
 50: 'warn_slippery_road',
 51: 'warn_speed_bumper',
 52: 'warn_traffic_light',
 53: 'warn_tram',
 54: 'warn_two_way_traffic',
 55: 'warn_wild_animals'}

In [23]:
def yolo_to_coordinates(yolo_annotation, img_width=640, img_height=640):
    """
    Convert YOLO annotation to (xmin, ymin, xmax, ymax) coordinates.
    
    :param yolo_annotation: A list containing [class_id, x_center, y_center, width, height, confidence]
    :param img_width: Width of the image
    :param img_height: Height of the image
    :return: A tuple (class_id, xmin, ymin, xmax, ymax, confidence)
    """
    class_id, x_center, y_center, width, height, confidence = yolo_annotation
    xmin = (x_center - width / 2) * img_width
    ymin = (y_center - height / 2) * img_height
    xmax = (x_center + width / 2) * img_width
    ymax = (y_center + height / 2) * img_height
    return class_id, xmin, ymin, xmax, ymax, confidence

In [4]:
def yolo_to_coordinates_curation(yolo_annotation, img_width=640, img_height=640):
    """
    Convert YOLO annotation to (xmin, ymin, xmax, ymax) coordinates.
    
    :param yolo_annotation: A list containing [class_id, x_center, y_center, width, height, confidence]
    :param img_width: Width of the image
    :param img_height: Height of the image
    :return: A tuple (class_id, xmin, ymin, xmax, ymax, confidence)
    """
    class_id, x_center, y_center, width, height = yolo_annotation
    xmin = (x_center - width / 2) * img_width
    ymin = (y_center - height / 2) * img_height
    xmax = (x_center + width / 2) * img_width
    ymax = (y_center + height / 2) * img_height
    return class_id, xmin, ymin, xmax, ymax

In [5]:
def process_yolo_files(folder_path, img_width=640, img_height=640):
    """
    Process YOLO annotation files in a folder and create a DataFrame.
    
    :param folder_path: Path to the folder containing YOLO annotation files
    :param img_width: Width of the image
    :param img_height: Height of the image
    :return: DataFrame with columns ['file_name', 'label', 'xmin', 'ymin', 'xmax', 'ymax', 'confidence']
    """
    data = []
    
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.txt'):
            file_path = os.path.join(folder_path, file_name)
            with open(file_path, 'r') as file:
                for line in file:
                    parts = line.strip().split()
                    annotation = list(map(float, parts))
                    converted_annotation = yolo_to_coordinates(annotation, img_width, img_height)
                    data.append([file_name] + list(converted_annotation))
    
    df = pd.DataFrame(data, columns=['file_name', 'label', 'xmin', 'ymin', 'xmax', 'ymax', 'confidence'])
    return df

In [6]:
def process_yolo_files_curation(folder_path, img_width=640, img_height=640):
    """
    Process YOLO annotation files in a folder and create a DataFrame.
    
    :param folder_path: Path to the folder containing YOLO annotation files
    :param img_width: Width of the image
    :param img_height: Height of the image
    :return: DataFrame with columns ['file_name', 'label', 'xmin', 'ymin', 'xmax', 'ymax', 'confidence']
    """
    data = []
    
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.txt'):
            file_path = os.path.join(folder_path, file_name)
            with open(file_path, 'r') as file:
                for line in file:
                    parts = line.strip().split()
                    annotation = list(map(float, parts))
                    converted_annotation = yolo_to_coordinates_curation(annotation, img_width, img_height)
                    data.append([file_name] + list(converted_annotation))
    
    df = pd.DataFrame(data, columns=['file_name', 'label', 'xmin', 'ymin', 'xmax', 'ymax'])
    return df

In [7]:
def bbox_to_polygon(bbox):
    """
    Convert bounding box to Shapely Polygon.
    
    :param bbox: List containing [xmin, ymin, xmax, ymax]
    :return: Shapely Polygon object
    """
    xmin, ymin, xmax, ymax = bbox
    # Define the four corners of the bounding box
    points = [(xmin, ymin), (xmax, ymin), (xmax, ymax), (xmin, ymax)]
    # Create and return a Shapely Polygon
    return Polygon(points)

In [8]:
# return overlap percentage
def get_overlap_percentage(poly1_cords, poly2_cords):
    poly1 = bbox_to_polygon(poly1_cords)
    poly2 = bbox_to_polygon(poly2_cords)
    intersection_area = poly1.intersection(poly2).area
    union_area = poly1.union(poly2).area
    overlap_percentage = intersection_area / union_area
    return overlap_percentage

In [9]:
[list(curation_df[['xmin','ymin','xmax','ymax']].values[0])]

NameError: name 'curation_df' is not defined

In [11]:
get_overlap_percentage(curation_df[['xmin','ymin','xmax','ymax']].values[0], v1_folder_path[['xmin','ymin','xmax','ymax']].values[1])

NameError: name 'curation_df' is not defined

# Curation

In [12]:
curation_folder_path = r'D:\Projects\Interview\phonologies\assignment\data\YOLO_Data\val\labels'
curation_df = process_yolo_files_curation(curation_folder_path)
curation_df.label = curation_df.label.astype(int)
curation_df.label = curation_df.label.map(yolo_part_mapping)
print(curation_df.shape)
curation_df.head(2)

(874, 6)


,file_name,label,xmin,ymin,xmax,ymax
0,SNAG-00013_png_jpg.rf.5655f1d044e0f5cccc6cf3c1...,mand_roundabout,81.0,309.0,178.0,399.0
1,SNAG-00014_png_jpg.rf.69143aca9eb36598083bb8c9...,mand_roundabout,462.0,271.0,533.0,347.0


In [13]:
print('Number of images in the curation folder:', curation_df.file_name.nunique())

Number of images in the curation folder: 500


In [14]:
curation_df.label.value_counts()

label
prio_give_way            148
forb_stopping            126
info_crosswalk            80
mand_roundabout           70
prio_stop                 66
prio_priority_road        63
forb_ahead                47
mand_right                29
mand_pass_left_right      22
info_taxi_parking         18
forb_right                16
mand_pass_right           15
forb_left                 14
info_parking              14
forb_speed_over_20        12
info_one_way_traffic      12
forb_speed_over_30        12
warn_construction         11
mand_left                 10
mand_straight              9
warn_tram                  9
mand_straight_right        9
forb_u_turn                7
mand_pass_left             7
warn_children              6
info_bus_station           5
warn_two_way_traffic       5
warn_cyclists              5
forb_weight_over_3.5t      4
warn_traffic_light         4
forb_overtake              4
warn_speed_bumper          4
warn_roundabout            2
forb_speed_over_5          2
forb_tru

# V1 Model

In [5]:
v1_folder_path = r'D:\Projects\Interview\phonologies\assignment\data\Prediction\V1_model_output\labels'
v1_df = process_yolo_files(v1_folder_path)
v1_df.label = v1_df.label.astype(int)
v1_df.label = v1_df.label.map(yolo_part_mapping)
print(v1_df.shape)
v1_df.head(2)

NameError: name 'process_yolo_files' is not defined

In [17]:
v1_df[['file_name', 'label']].value_counts()

file_name                                                      label               
SNAG12-RADU_884_png.rf.5429cd7610a7efa041c966485736ea2e.txt    forb_stopping           3
SNAG12-RADU_931_png.rf.5b0326367a36b7aa64d63be89cfe3f24.txt    info_crosswalk          3
SNAG12-RADU_882_png.rf.9beb8b123d09ebeeef3d59ca5739a120.txt    prio_give_way           3
SNAG12-RADU_884_png.rf.5429cd7610a7efa041c966485736ea2e.txt    info_one_way_traffic    2
SNAGDEEA-0213_png_jpg.rf.acd42213034993a582cb2f8ecf56e96b.txt  info_crosswalk          2
                                                                                      ..
SNAG12-RADU_865_png.rf.f28394b805aa8371258dc8c8e49f82ef.txt    mand_straight           1
SNAG12-RADU_866_png.rf.fdcc28fffc7910a69bfa1b58117cacae.txt    info_parking            1
                                                               mand_right              1
                                                               mand_straight           1
SNAGDEEA-0245_png_jpg.rf.3

In [18]:
v1_df[v1_df['file_name'] == 'SNAG12-RADU_884_png.rf.5429cd7610a7efa041c966485736ea2e.txt']

,file_name,label,xmin,ymin,xmax,ymax,confidence
363,SNAG12-RADU_884_png.rf.5429cd7610a7efa041c9664...,forb_stopping,206.00000,329.99968,226.00000,351.99968,0.519227
364,SNAG12-RADU_884_png.rf.5429cd7610a7efa041c9664...,mand_straight_right,567.99984,299.00000,588.99984,321.00000,0.632269
365,SNAG12-RADU_884_png.rf.5429cd7610a7efa041c9664...,forb_stopping,569.00032,321.99968,589.00032,343.99968,0.761715
366,SNAG12-RADU_884_png.rf.5429cd7610a7efa041c9664...,info_one_way_traffic,205.99984,307.99968,226.99984,329.99968,0.933844
367,SNAG12-RADU_884_png.rf.5429cd7610a7efa041c9664...,forb_stopping,376.00016,321.00000,423.00016,371.00000,0.938998
368,SNAG12-RADU_884_png.rf.5429cd7610a7efa041c9664...,info_one_way_traffic,373.00000,271.00016,423.00000,320.00016,0.941434
369,SNAG12-RADU_884_png.rf.5429cd7610a7efa041c9664...,prio_give_way,361.00032,207.99968,429.00032,269.99968,0.942490


In [62]:
V1_accuracy_calculation = pd.DataFrame()
# calculate the accuracy
for each_name in curation_df.file_name.unique():
    curr_curation_df = curation_df[curation_df.file_name == each_name]
    curr_v1_df = v1_df[v1_df.file_name == each_name]
    for each_label in curr_curation_df.label.unique():
        curated_bbox = list(curr_curation_df[curr_curation_df.label == each_label][['xmin','ymin','xmax','ymax']].values[0])
        try:
            v1_bbox = list(curr_v1_df[curr_v1_df.label == each_label][['xmin','ymin','xmax','ymax']].values[0])
            overlap = get_overlap_percentage(curated_bbox, v1_bbox)
            # V1_accuracy_calculation = pd.concat([V1_accuracy_calculation, pd.DataFrame({'file_name': each_name, 'label': each_label, 'overlap': overlap})], ignore_index=True)
            new_row = pd.DataFrame({'file_name': [each_name], 'label': [each_label], 'overlap': [overlap]})

            V1_accuracy_calculation = pd.concat([V1_accuracy_calculation, new_row], ignore_index=True)
        except Exception as e:
            print(e)
        break

index 0 is out of bounds for axis 0 with size 0
index 0 is out of bounds for axis 0 with size 0
index 0 is out of bounds for axis 0 with size 0
index 0 is out of bounds for axis 0 with size 0
index 0 is out of bounds for axis 0 with size 0
index 0 is out of bounds for axis 0 with size 0
index 0 is out of bounds for axis 0 with size 0
index 0 is out of bounds for axis 0 with size 0
index 0 is out of bounds for axis 0 with size 0
index 0 is out of bounds for axis 0 with size 0
index 0 is out of bounds for axis 0 with size 0
index 0 is out of bounds for axis 0 with size 0
index 0 is out of bounds for axis 0 with size 0
index 0 is out of bounds for axis 0 with size 0
index 0 is out of bounds for axis 0 with size 0
index 0 is out of bounds for axis 0 with size 0
index 0 is out of bounds for axis 0 with size 0
index 0 is out of bounds for axis 0 with size 0
index 0 is out of bounds for axis 0 with size 0


In [63]:
V1_accuracy_calculation

,file_name,label,overlap
0,SNAG-00013_png_jpg.rf.5655f1d044e0f5cccc6cf3c1...,mand_roundabout,0.919562
1,SNAG-00014_png_jpg.rf.69143aca9eb36598083bb8c9...,mand_roundabout,0.947635
2,SNAG-00015_png_jpg.rf.62a9e36011a59c52ce76c306...,mand_roundabout,0.966060
3,SNAG-00016_png_jpg.rf.a20b6a9a3f604f1d619e18b7...,forb_stopping,0.947965
4,SNAG-00017_png_jpg.rf.a4ece30ed6fd85ecc65ad4e8...,forb_stopping,0.941976
...,...,...,...
476,SNAGDEEA-0240_png_jpg.rf.0c689c27447803d32dcb0...,prio_stop,0.974125
477,SNAGDEEA-0241_png_jpg.rf.4d41908e7628ce4111a50...,prio_stop,0.980964
478,SNAGDEEA-0242_png_jpg.rf.398df451e94ce4314d4ca...,prio_give_way,0.906883
479,SNAGDEEA-0243_png_jpg.rf.3bc4ff5873d190933b14b...,prio_give_way,0.938532


In [6]:
import os
import numpy as np

def load_labels(folder):
    labels = {}
    for filename in os.listdir(folder):
        if filename.endswith('.txt'):
            filepath = os.path.join(folder, filename)
            with open(filepath, 'r') as f:
                lines = f.readlines()
                image_labels = []
                for line in lines:
                    parts = list(map(float, line.strip().split()))
                    image_labels.append(parts)
                labels[filename] = image_labels
    return labels

def iou(box1, box2):
    # box format: [class, x_center, y_center, width, height]
    x1_min = box1[1] - box1[3] / 2
    x1_max = box1[1] + box1[3] / 2
    y1_min = box1[2] - box1[4] / 2
    y1_max = box1[2] + box1[4] / 2
    
    x2_min = box2[1] - box2[3] / 2
    x2_max = box2[1] + box2[3] / 2
    y2_min = box2[2] - box2[4] / 2
    y2_max = box2[2] + box2[4] / 2
    
    inter_x_min = max(x1_min, x2_min)
    inter_x_max = min(x1_max, x2_max)
    inter_y_min = max(y1_min, y2_min)
    inter_y_max = min(y1_max, y2_max)
    
    if inter_x_max < inter_x_min or inter_y_max < inter_y_min:
        return 0.0
    
    intersection = (inter_x_max - inter_x_min) * (inter_y_max - inter_y_min)
    box1_area = (x1_max - x1_min) * (y1_max - y1_min)
    box2_area = (x2_max - x2_min) * (y2_max - y2_min)
    
    union = box1_area + box2_area - intersection
    return intersection / union

def compute_tp_fp_fn(pred_labels, gt_labels, iou_threshold=0.5):
    tp = 0
    fp = 0
    fn = 0
    
    matched_gt_indices = set()
    
    for pred_box in pred_labels:
        best_iou = 0
        best_gt_index = -1
        for gt_index, gt_box in enumerate(gt_labels):
            if gt_box[0] == pred_box[0]:  # same class
                current_iou = iou(pred_box, gt_box)
                if current_iou > best_iou:
                    best_iou = current_iou
                    best_gt_index = gt_index
        if best_iou > iou_threshold:
            if best_gt_index not in matched_gt_indices:
                tp += 1
                matched_gt_indices.add(best_gt_index)
            else:
                fp += 1
        else:
            fp += 1
    
    fn = len(gt_labels) - len(matched_gt_indices)
    
    return tp, fp, fn

# Loading prediction and ground truth labels
prediction_folder = r'''D:/Projects/Interview/phonologies/assignment/data/Prediction/V1_model_output/labels/'''
ground_truth_folder = r'''D:/Projects/Interview/phonologies/assignment/data/YOLO_Data/val/labels/'''

pred_labels = load_labels(prediction_folder)
gt_labels = load_labels(ground_truth_folder)


In [3]:
from collections import defaultdict

def compute_precision_recall(tp, fp, fn):
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    return precision, recall

def compute_map(pred_labels, gt_labels, iou_threshold=0.5):
    class_tp_fp_fn = defaultdict(lambda: [0, 0, 0])
    
    for image_id in pred_labels:
        preds = pred_labels[image_id]
        gts = gt_labels.get(image_id, [])
        
        tp, fp, fn = compute_tp_fp_fn(preds, gts, iou_threshold)
        
        for pred in preds:
            class_tp_fp_fn[pred[0]][0] += tp
            class_tp_fp_fn[pred[0]][1] += fp
            class_tp_fp_fn[pred[0]][2] += fn
            
        for gt in gts:
            if gt[0] not in [pred[0] for pred in preds]:
                class_tp_fp_fn[gt[0]][2] += 1
    
    precision_recall_map = {}
    for class_id, (tp, fp, fn) in class_tp_fp_fn.items():
        precision, recall = compute_precision_recall(tp, fp, fn)
        precision_recall_map[class_id] = (precision, recall)
    
    return precision_recall_map

precision_recall_map = compute_map(pred_labels, gt_labels)

for class_id, (precision, recall) in precision_recall_map.items():
    print(f"Class {class_id}: Precision = {precision:.2f}, Recall = {recall:.2f}")

# To calculate mAP, we would typically use the precision-recall curve and calculate the area under the curve.
# Here we'll compute a simple average of precisions as an approximation.

map_score = np.mean([precision for precision, recall in precision_recall_map.values()])
print(f"mAP: {map_score:.2f}")

Class 35.0: Precision = 0.89, Recall = 0.96
Class 23.0: Precision = 0.88, Recall = 0.90
Class 17.0: Precision = 0.88, Recall = 0.92
Class 1.0: Precision = 0.91, Recall = 0.97
Class 39.0: Precision = 0.86, Recall = 0.93
Class 32.0: Precision = 0.79, Recall = 0.86
Class 41.0: Precision = 0.77, Recall = 0.95
Class 34.0: Precision = 0.81, Recall = 0.88
Class 29.0: Precision = 0.50, Recall = 0.23
Class 25.0: Precision = 0.79, Recall = 0.91
Class 26.0: Precision = 0.75, Recall = 0.77
Class 2.0: Precision = 0.76, Recall = 0.81
Class 4.0: Precision = 0.86, Recall = 0.86
Class 19.0: Precision = 0.79, Recall = 0.79
Class 40.0: Precision = 0.89, Recall = 0.89
Class 9.0: Precision = 0.82, Recall = 0.82
Class 12.0: Precision = 0.50, Recall = 0.86
Class 20.0: Precision = 1.00, Recall = 0.71
Class 10.0: Precision = 0.50, Recall = 0.75
Class 47.0: Precision = 0.75, Recall = 1.00
Class 33.0: Precision = 0.70, Recall = 0.84
Class 3.0: Precision = 0.76, Recall = 0.94
Class 27.0: Precision = 0.78, Recall 

In [5]:
import pandas as pd

In [17]:
V1_score_df = pd.DataFrame(precision_recall_map)
print(V1_score_df.shape)
V1_score_df.head(2)

(2, 45)


,35.0,23.0,17.0,1.0,39.0,32.0,41.0,34.0,29.0,25.0,...,53.0,31.0,45.0,49.0,48.0,55.0,51.0,21.0,28.0,44.0
0,0.892617,0.875940,0.882155,0.905405,0.856383,0.788235,0.767857,0.808333,0.500000,0.792208,...,0.892857,0.555556,0.8,1.0,0.333333,0.333333,0.909091,0.5,0.571429,1.0
1,0.963768,0.903101,0.919298,0.971014,0.933333,0.858974,0.948529,0.881818,0.230769,0.910448,...,0.862069,0.454545,1.0,1.0,0.500000,0.500000,1.000000,1.0,1.000000,1.0


In [18]:
V1_score_df = V1_score_df.T.reset_index()
V1_score_df.columns = ['label', 'precision', 'recall']
V1_score_df['label_parts'] = V1_score_df.label.map(yolo_part_mapping)
V1_score_df['F_score'] = 2 * V1_score_df['precision'] * V1_score_df['recall'] / (V1_score_df['precision'] + V1_score_df['recall'])
V1_score_df.head(2)

,label,precision,recall,label_parts,F_score
0,35.0,0.892617,0.963768,mand_roundabout,0.926829
1,23.0,0.875940,0.903101,info_crosswalk,0.889313


In [19]:
V1_score_df.sort_values(by='F_score', ascending=False).head(10)

,label,precision,recall,label_parts,F_score
44,44.0,1.000000,1.000000,warn_crosswalk,1.000000
38,49.0,1.000000,1.000000,warn_roundabout,1.000000
29,42.0,1.000000,0.944444,warn_children,0.971429
41,51.0,0.909091,1.000000,warn_speed_bumper,0.952381
3,1.0,0.905405,0.971014,forb_ahead,0.937063
0,35.0,0.892617,0.963768,mand_roundabout,0.926829
2,17.0,0.882155,0.919298,forb_stopping,0.900344
30,8.0,0.923077,0.878049,forb_speed_over_20,0.900000
4,39.0,0.856383,0.933333,prio_give_way,0.893204
14,40.0,0.894737,0.885417,prio_priority_road,0.890052


In [14]:
import os
import numpy as np
from collections import defaultdict

def load_labels(folder):
    labels = {}
    for filename in os.listdir(folder):
        if filename.endswith('.txt'):
            filepath = os.path.join(folder, filename)
            with open(filepath, 'r') as f:
                lines = f.readlines()
                image_labels = []
                for line in lines:
                    parts = list(map(float, line.strip().split()))
                    image_labels.append(parts)
                labels[filename] = image_labels
    return labels

def iou(box1, box2):
    # box format: [class, x_center, y_center, width, height]
    x1_min = box1[1] - box1[3] / 2
    x1_max = box1[1] + box1[3] / 2
    y1_min = box1[2] - box1[4] / 2
    y1_max = box1[2] + box1[4] / 2
    
    x2_min = box2[1] - box2[3] / 2
    x2_max = box2[1] + box2[3] / 2
    y2_min = box2[2] - box2[4] / 2
    y2_max = box2[2] + box2[4] / 2
    
    inter_x_min = max(x1_min, x2_min)
    inter_x_max = min(x1_max, x2_max)
    inter_y_min = max(y1_min, y2_min)
    inter_y_max = min(y1_max, y2_max)
    
    if inter_x_max < inter_x_min or inter_y_max < inter_y_min:
        return 0.0
    
    intersection = (inter_x_max - inter_x_min) * (inter_y_max - inter_y_min)
    box1_area = (x1_max - x1_min) * (y1_max - y1_min)
    box2_area = (x2_max - x2_min) * (y2_max - y2_min)
    
    union = box1_area + box2_area - intersection
    return intersection / union

def compute_tp_fp_fn(pred_labels, gt_labels, iou_threshold=0.5):
    tp = 0
    fp = 0
    fn = 0
    matched_gt_indices = set()
    
    for pred_box in pred_labels:
        best_iou = 0
        best_gt_index = -1
        for gt_index, gt_box in enumerate(gt_labels):
            if gt_box[0] == pred_box[0]:  # same class
                current_iou = iou(pred_box, gt_box)
                if current_iou > best_iou:
                    best_iou = current_iou
                    best_gt_index = gt_index
        if best_iou > iou_threshold:
            if best_gt_index not in matched_gt_indices:
                tp += 1
                matched_gt_indices.add(best_gt_index)
            else:
                fp += 1
        else:
            fp += 1
    
    fn = len(gt_labels) - len(matched_gt_indices)
    
    return tp, fp, fn

def compute_precision_recall(tp, fp, fn):
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    return precision, recall

def compute_f1(precision, recall):
    if precision + recall == 0:
        return 0
    return 2 * (precision * recall) / (precision + recall)

def compute_metrics(pred_labels, gt_labels, iou_threshold=0.5):
    class_stats = defaultdict(lambda: {'tp': 0, 'fp': 0, 'fn': 0, 'count': 0, 'pred_count': 0})
    
    for image_id in pred_labels:
        preds = pred_labels[image_id]
        gts = gt_labels.get(image_id, [])
        
        tp, fp, fn = compute_tp_fp_fn(preds, gts, iou_threshold)
        
        for pred in preds:
            class_stats[pred[0]]['pred_count'] += 1
        
        for gt in gts:
            class_stats[gt[0]]['count'] += 1
            
        for pred in preds:
            class_stats[pred[0]]['tp'] += tp
            class_stats[pred[0]]['fp'] += fp
            class_stats[pred[0]]['fn'] += fn
    
    for class_id, stats in class_stats.items():
        tp, fp, fn = stats['tp'], stats['fp'], stats['fn']
        precision, recall = compute_precision_recall(tp, fp, fn)
        f1 = compute_f1(precision, recall)
        stats['precision'] = precision
        stats['recall'] = recall
        stats['f1'] = f1
    
    return class_stats

# Loading prediction and ground truth labels
prediction_folder = r'''D:/Projects/Interview/phonologies/assignment/data/Prediction/V1_model_output/labels/'''
ground_truth_folder = r'''D:/Projects/Interview/phonologies/assignment/data/YOLO_Data/val/labels/'''

pred_labels = load_labels(prediction_folder)
gt_labels = load_labels(ground_truth_folder)

class_stats = compute_metrics(pred_labels, gt_labels)

# Print the results in a structured format
print(f"{'label':<10} {'precision':<10} {'recall':<10} {'f1':<10} {'count':<10} {'pred_count':<15} {'tp':<5} {'fp':<5} {'fn':<5}")
for class_id, stats in class_stats.items():
    print(f"{int(class_id):<10} {stats['precision']:<10.2f} {stats['recall']:<10.2f} {stats['f1']:<10.2f} {stats['count']:<10} {stats['pred_count']:<15} {stats['tp']:<5} {stats['fp']:<5} {stats['fn']:<5}")


label      precision  recall     f1         count      pred_count      tp    fp    fn   
35         0.89       0.98       0.93       70         69              133   16    3    
23         0.88       0.91       0.89       80         86              233   33    23   
17         0.88       0.94       0.91       124        122             262   35    18   
1          0.91       1.00       0.95       47         46              67    7     0    
39         0.86       0.93       0.89       148        154             322   54    23   
32         0.79       0.86       0.82       22         29              67    18    11   
41         0.77       0.95       0.85       66         81              129   39    7    
34         0.81       0.88       0.84       29         41              97    23    13   
29         0.50       1.00       0.67       10         2               3     3     0    
25         0.79       0.91       0.85       12         19              61    16    6    
26         0.75      

In [ ]:
load_labels(ground_truth_folder)

In [15]:
V1_score_df = pd.DataFrame(class_stats)
V1_score_df = V1_score_df.T.reset_index()
V1_score_df.columns = ['class_id', 'tp', 'fp', 'fn', 'count', 'pred_count', 'precision', 'recall', 'f1']
V1_score_df['class_label'] = V1_score_df['class_id'].map(yolo_part_mapping)
V1_score_df.shape

(45, 10)

In [16]:
V1_score_df

,class_id,tp,fp,fn,count,pred_count,precision,recall,f1,class_label
0,35.0,133.0,16.0,3.0,70.0,69.0,0.892617,0.977941,0.933333,mand_roundabout
1,23.0,233.0,33.0,23.0,80.0,86.0,0.875940,0.910156,0.892720,info_crosswalk
2,17.0,262.0,35.0,18.0,124.0,122.0,0.882155,0.935714,0.908146,forb_stopping
3,1.0,67.0,7.0,0.0,47.0,46.0,0.905405,1.000000,0.950355,forb_ahead
4,39.0,322.0,54.0,23.0,148.0,154.0,0.856383,0.933333,0.893204,prio_give_way
5,32.0,67.0,18.0,11.0,22.0,29.0,0.788235,0.858974,0.822086,mand_pass_left_right
6,41.0,129.0,39.0,7.0,66.0,81.0,0.767857,0.948529,0.848684,prio_stop
7,34.0,97.0,23.0,13.0,29.0,41.0,0.808333,0.881818,0.843478,mand_right
8,29.0,3.0,3.0,0.0,10.0,2.0,0.500000,1.000000,0.666667,mand_left
9,25.0,61.0,16.0,6.0,12.0,19.0,0.792208,0.910448,0.847222,info_one_way_traffic


In [19]:
import os
import numpy as np
from collections import defaultdict
import pandas as pd

def load_labels(folder):
    labels = {}
    for filename in os.listdir(folder):
        if filename.endswith('.txt'):
            filepath = os.path.join(folder, filename)
            with open(filepath, 'r') as f:
                lines = f.readlines()
                image_labels = []
                for line in lines:
                    parts = list(map(float, line.strip().split()))
                    image_labels.append(parts)
                labels[filename] = image_labels
    return labels

def iou(box1, box2):
    x1_min = box1[1] - box1[3] / 2
    x1_max = box1[1] + box1[3] / 2
    y1_min = box1[2] - box1[4] / 2
    y1_max = box1[2] + box1[4] / 2
    
    x2_min = box2[1] - box2[3] / 2
    x2_max = box2[1] + box2[3] / 2
    y2_min = box2[2] - box2[4] / 2
    y2_max = box2[2] + box2[4] / 2
    
    inter_x_min = max(x1_min, x2_min)
    inter_x_max = min(x1_max, x2_max)
    inter_y_min = max(y1_min, y2_min)
    inter_y_max = min(y1_max, y2_max)
    
    if inter_x_max < inter_x_min or inter_y_max < inter_y_min:
        return 0.0
    
    intersection = (inter_x_max - inter_x_min) * (inter_y_max - inter_y_min)
    box1_area = (x1_max - x1_min) * (y1_max - y1_min)
    box2_area = (x2_max - x2_min) * (y2_max - y2_min)
    
    union = box1_area + box2_area - intersection
    return intersection / union

def compute_tp_fp_fn(pred_labels, gt_labels, iou_threshold=0.5):
    tp = 0
    fp = 0
    fn = 0
    matched_gt_indices = set()
    
    for pred_box in pred_labels:
        best_iou = 0
        best_gt_index = -1
        for gt_index, gt_box in enumerate(gt_labels):
            if gt_box[0] == pred_box[0]:  # same class
                current_iou = iou(pred_box, gt_box)
                if current_iou > best_iou:
                    best_iou = current_iou
                    best_gt_index = gt_index
        if best_iou > iou_threshold:
            if best_gt_index not in matched_gt_indices:
                tp += 1
                matched_gt_indices.add(best_gt_index)
            else:
                fp += 1
        else:
            fp += 1
    
    fn = len(gt_labels) - len(matched_gt_indices)
    
    return tp, fp, fn

def compute_precision_recall(tp, fp, fn):
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    return precision, recall

def compute_f1(precision, recall):
    if precision + recall == 0:
        return 0
    return 2 * (precision * recall) / (precision + recall)

def compute_metrics(pred_labels, gt_labels, iou_threshold=0.5):
    class_stats = defaultdict(lambda: {'tp': 0, 'fp': 0, 'fn': 0, 'count': 0, 'pred_count': 0})
    
    for image_id in pred_labels:
        preds = pred_labels[image_id]
        gts = gt_labels.get(image_id, [])
        
        tp, fp, fn = compute_tp_fp_fn(preds, gts, iou_threshold)
        
        for pred in preds:
            class_stats[pred[0]]['pred_count'] += 1
        
        for gt in gts:
            class_stats[gt[0]]['count'] += 1  # Curated count is updated here
            
        for pred in preds:
            class_stats[pred[0]]['tp'] += tp
            class_stats[pred[0]]['fp'] += fp
            class_stats[pred[0]]['fn'] += fn
    
    for class_id, stats in class_stats.items():
        tp, fp, fn = stats['tp'], stats['fp'], stats['fn']
        precision, recall = compute_precision_recall(tp, fp, fn)
        f1 = compute_f1(precision, recall)
        stats['precision'] = precision
        stats['recall'] = recall
        stats['f1'] = f1
    
    return class_stats

# Loading prediction and ground truth labels
prediction_folder = r'''D:/Projects/Interview/phonologies/assignment/data/Prediction/V2_model_output/labels/'''
ground_truth_folder = r'''D:/Projects/Interview/phonologies/assignment/data/YOLO_Data/val/labels/'''

pred_labels = load_labels(prediction_folder)
gt_labels = load_labels(ground_truth_folder)

class_stats = compute_metrics(pred_labels, gt_labels)

# Print the results in a structured format
print(f"{'label':<10} {'precision':<10} {'recall':<10} {'f1':<10} {'count':<10} {'curated_count':<15} {'pred_count':<15} {'tp':<5} {'fp':<5} {'fn':<5}")
for class_id, stats in class_stats.items():
    print(f"{int(class_id):<10} {stats['precision']:<10.2f} {stats['recall']:<10.2f} {stats['f1']:<10.2f} {stats['count']:<10} {stats['count']:<15} {stats['pred_count']:<15} {stats['tp']:<5} {stats['fp']:<5} {stats['fn']:<5}")

# Create DataFrame
V1_score_df = pd.DataFrame(class_stats).T.reset_index()
V1_score_df.columns = ['class_id', 'tp', 'fp', 'fn', 'curated_count', 'pred_count', 'precision', 'recall', 'f1']
V1_score_df['class_label'] = V1_score_df['class_id'].map(yolo_part_mapping)
V1_score_df

label      precision  recall     f1         count      curated_count   pred_count      tp    fp    fn   
35         0.88       0.98       0.93       70         70              71              135   18    3    
23         0.86       0.94       0.90       80         80              91              249   40    17   
17         0.84       0.93       0.88       125        125             128             276   52    20   
1          0.83       0.97       0.89       47         47              50              68    14    2    
25         0.80       0.89       0.84       12         12              19              67    17    8    
26         0.79       0.90       0.84       14         14              18              44    12    5    
39         0.87       0.94       0.91       148        148             153             324   47    21   
32         0.88       0.92       0.90       22         22              25              66    9     6    
41         0.82       0.94       0.88       66         

,class_id,tp,fp,fn,curated_count,pred_count,precision,recall,f1,class_label
0,35.0,135.0,18.0,3.0,70.0,71.0,0.882353,0.978261,0.927835,mand_roundabout
1,23.0,249.0,40.0,17.0,80.0,91.0,0.861592,0.936090,0.897297,info_crosswalk
2,17.0,276.0,52.0,20.0,125.0,128.0,0.841463,0.932432,0.884615,forb_stopping
3,1.0,68.0,14.0,2.0,47.0,50.0,0.829268,0.971429,0.894737,forb_ahead
4,25.0,67.0,17.0,8.0,12.0,19.0,0.797619,0.893333,0.842767,info_one_way_traffic
5,26.0,44.0,12.0,5.0,14.0,18.0,0.785714,0.897959,0.838095,info_parking
6,39.0,324.0,47.0,21.0,148.0,153.0,0.873315,0.939130,0.905028,prio_give_way
7,32.0,66.0,9.0,6.0,22.0,25.0,0.880000,0.916667,0.897959,mand_pass_left_right
8,41.0,118.0,26.0,7.0,66.0,73.0,0.819444,0.944000,0.877323,prio_stop
9,34.0,78.0,17.0,9.0,29.0,36.0,0.821053,0.896552,0.857143,mand_right


# 12 June

In [24]:
import os
import numpy as np
from collections import defaultdict
import pandas as pd

def load_labels(folder):
    labels = {}
    for filename in os.listdir(folder):
        if filename.endswith('.txt'):
            filepath = os.path.join(folder, filename)
            with open(filepath, 'r') as f:
                lines = f.readlines()
                image_labels = []
                for line in lines:
                    parts = list(map(float, line.strip().split()))
                    image_labels.append(parts)
                labels[filename] = image_labels
    return labels

def iou(box1, box2):
    x1_min = box1[1] - box1[3] / 2
    x1_max = box1[1] + box1[3] / 2
    y1_min = box1[2] - box1[4] / 2
    y1_max = box1[2] + box1[4] / 2
    
    x2_min = box2[1] - box2[3] / 2
    x2_max = box2[1] + box2[3] / 2
    y2_min = box2[2] - box2[4] / 2
    y2_max = box2[2] + box2[4] / 2
    
    inter_x_min = max(x1_min, x2_min)
    inter_x_max = min(x1_max, x2_max)
    inter_y_min = max(y1_min, y2_min)
    inter_y_max = min(y1_max, y2_max)
    
    if inter_x_max < inter_x_min or inter_y_max < inter_y_min:
        return 0.0
    
    intersection = (inter_x_max - inter_x_min) * (inter_y_max - inter_y_min)
    box1_area = (x1_max - x1_min) * (y1_max - y1_min)
    box2_area = (x2_max - x2_min) * (y2_max - y2_min)
    
    union = box1_area + box2_area - intersection
    return intersection / union

def compute_tp_fp_fn(pred_labels, gt_labels, iou_threshold=0.5):
    tp = 0
    fp = 0
    fn = 0
    matched_gt_indices = set()
    
    for pred_box in pred_labels:
        best_iou = 0
        best_gt_index = -1
        for gt_index, gt_box in enumerate(gt_labels):
            if gt_box[0] == pred_box[0]:  # same class
                current_iou = iou(pred_box, gt_box)
                if current_iou > best_iou:
                    best_iou = current_iou
                    best_gt_index = gt_index
        if best_iou > iou_threshold:
            if best_gt_index not in matched_gt_indices:
                tp += 1
                matched_gt_indices.add(best_gt_index)
            else:
                fp += 1
        else:
            fp += 1
    
    fn = len(gt_labels) - len(matched_gt_indices)
    
    return tp, fp, fn

def compute_precision_recall(tp, fp, fn):
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    return precision, recall

def compute_f1(precision, recall):
    if precision + recall == 0:
        return 0
    return 2 * (precision * recall) / (precision + recall)

def compute_metrics(pred_labels, gt_labels, iou_threshold=0.5):
    class_stats = defaultdict(lambda: {'tp': 0, 'fp': 0, 'fn': 0, 'curate_count': 0, 'pred_count': 0})
    
    for image_id in pred_labels:
        preds = pred_labels[image_id]
        gts = gt_labels.get(image_id, [])
        
        tp, fp, fn = compute_tp_fp_fn(preds, gts, iou_threshold)
        
        for pred in preds:
            class_stats[pred[0]]['pred_count'] += 1
        
        for gt in gts:
            class_stats[gt[0]]['curate_count'] += 1
            
        for class_id, stats in class_stats.items():
            stats['tp'] += tp
            stats['fp'] += fp
            stats['fn'] += fn
    
    results = []
    for class_id, stats in class_stats.items():
        tp, fp, fn = stats['tp'], stats['fp'], stats['fn']
        precision, recall = compute_precision_recall(tp, fp, fn)
        f1 = compute_f1(precision, recall)
        results.append({
            'class_label': int(class_id),
            'Curate count': stats['curate_count'],
            'Pred count': stats['pred_count'],
            'TP': tp,
            'FP': fp,
            'FN': fn,
            'Precision': precision,
            'Recall': recall,
            'F1 Score': f1
        })
    
    return pd.DataFrame(results)

# Define folders
prediction_folder = r'''D:/Projects/Interview/phonologies/assignment/data/Prediction/V1_model_output/labels/'''
ground_truth_folder = r'''D:/Projects/Interview/phonologies/assignment/data/YOLO_Data/val/labels/'''

# Load labels
pred_labels = load_labels(prediction_folder)
gt_labels = load_labels(ground_truth_folder)

# Compute metrics
class_stats_df = compute_metrics(pred_labels, gt_labels)

# Print DataFrame
class_stats_df['class_label'] = class_stats_df['class_label'].map(yolo_part_mapping)
class_stats_df

,class_label,Curate count,Pred count,TP,FP,FN,Precision,Recall,F1 Score
0,mand_roundabout,70,69,807,135,62,0.856688,0.928654,0.891220
1,info_crosswalk,80,86,805,135,62,0.856383,0.928489,0.890980
2,forb_stopping,124,122,804,134,62,0.857143,0.928406,0.891353
3,forb_ahead,47,46,802,134,62,0.856838,0.928241,0.891111
4,prio_give_way,148,154,798,134,62,0.856223,0.927907,0.890625
5,mand_pass_left_right,22,29,795,134,62,0.855759,0.927655,0.890258
6,prio_stop,66,81,791,132,62,0.856988,0.927315,0.890766
7,mand_right,29,41,790,132,62,0.856833,0.927230,0.890643
8,mand_left,10,2,790,132,62,0.856833,0.927230,0.890643
9,info_one_way_traffic,12,19,777,126,60,0.860465,0.928315,0.893103


In [25]:
import os
import numpy as np
from collections import defaultdict
import pandas as pd

def load_labels(folder):
    labels = {}
    for filename in os.listdir(folder):
        if filename.endswith('.txt'):
            filepath = os.path.join(folder, filename)
            with open(filepath, 'r') as f:
                lines = f.readlines()
                image_labels = []
                for line in lines:
                    parts = list(map(float, line.strip().split()))
                    image_labels.append(parts)
                labels[filename] = image_labels
    return labels

def iou(box1, box2):
    x1_min = box1[1] - box1[3] / 2
    x1_max = box1[1] + box1[3] / 2
    y1_min = box1[2] - box1[4] / 2
    y1_max = box1[2] + box1[4] / 2
    
    x2_min = box2[1] - box2[3] / 2
    x2_max = box2[1] + box2[3] / 2
    y2_min = box2[2] - box2[4] / 2
    y2_max = box2[2] + box2[4] / 2
    
    inter_x_min = max(x1_min, x2_min)
    inter_x_max = min(x1_max, x2_max)
    inter_y_min = max(y1_min, y2_min)
    inter_y_max = min(y1_max, y2_max)
    
    if inter_x_max < inter_x_min or inter_y_max < inter_y_min:
        return 0.0
    
    intersection = (inter_x_max - inter_x_min) * (inter_y_max - inter_y_min)
    box1_area = (x1_max - x1_min) * (y1_max - y1_min)
    box2_area = (x2_max - x2_min) * (y2_max - y2_min)
    
    union = box1_area + box2_area - intersection
    return intersection / union

def compute_tp_fp_fn(pred_labels, gt_labels, iou_threshold=0.5):
    tp = 0
    fp = 0
    fn = 0
    matched_gt_indices = set()
    
    for pred_box in pred_labels:
        best_iou = 0
        best_gt_index = -1
        for gt_index, gt_box in enumerate(gt_labels):
            if gt_box[0] == pred_box[0]:  # same class
                current_iou = iou(pred_box, gt_box)
                if current_iou > best_iou:
                    best_iou = current_iou
                    best_gt_index = gt_index
        if best_iou > iou_threshold:
            if best_gt_index not in matched_gt_indices:
                tp += 1
                matched_gt_indices.add(best_gt_index)
            else:
                fp += 1
        else:
            fp += 1
    
    fn = len(gt_labels) - len(matched_gt_indices)
    
    return tp, fp, fn

def compute_precision_recall(tp, fp, fn):
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    return precision, recall

def compute_f1(precision, recall):
    if precision + recall == 0:
        return 0
    return 2 * (precision * recall) / (precision + recall)

def compute_metrics(pred_labels, gt_labels, iou_threshold=0.5):
    class_stats = defaultdict(lambda: {'tp': 0, 'fp': 0, 'fn': 0, 'curate_count': 0, 'pred_count': 0})
    
    for image_id in pred_labels:
        preds = pred_labels[image_id]
        gts = gt_labels.get(image_id, [])
        
        tp, fp, fn = compute_tp_fp_fn(preds, gts, iou_threshold)
        
        for pred in preds:
            class_stats[pred[0]]['pred_count'] += 1
        
        for gt in gts:
            class_stats[gt[0]]['curate_count'] += 1
            
        for class_id, stats in class_stats.items():
            stats['tp'] += tp
            stats['fp'] += fp
            stats['fn'] += fn
    
    results = []
    for class_id, stats in class_stats.items():
        tp, fp, fn = stats['tp'], stats['fp'], stats['fn']
        precision, recall = compute_precision_recall(tp, fp, fn)
        f1 = compute_f1(precision, recall)
        results.append({
            'class_label': int(class_id),
            'Curate count': stats['curate_count'],
            'Pred count': stats['pred_count'],
            'TP': tp,
            'FP': fp,
            'FN': fn,
            'Precision': precision,
            'Recall': recall,
            'F1 Score': f1
        })
    
    return pd.DataFrame(results)

# Define folders
prediction_folder = r'''D:/Projects/Interview/phonologies/assignment/data/Prediction/V2_model_output/labels/'''
ground_truth_folder = r'''D:/Projects/Interview/phonologies/assignment/data/YOLO_Data/val/labels/'''

# Load labels
pred_labels = load_labels(prediction_folder)
gt_labels = load_labels(ground_truth_folder)

# Compute metrics
class_stats_df = compute_metrics(pred_labels, gt_labels)

# Print DataFrame
class_stats_df['class_label'] = class_stats_df['class_label'].map(yolo_part_mapping)
class_stats_df

,class_label,Curate count,Pred count,TP,FP,FN,Precision,Recall,F1 Score
0,mand_roundabout,70,71,818,120,54,0.872068,0.938073,0.903867
1,info_crosswalk,80,91,816,120,54,0.871795,0.937931,0.903654
2,forb_stopping,125,128,815,119,54,0.872591,0.937860,0.904049
3,forb_ahead,47,50,813,119,54,0.872318,0.937716,0.903835
4,info_one_way_traffic,12,19,811,119,54,0.872043,0.937572,0.903621
5,info_parking,14,18,809,116,54,0.874595,0.937428,0.904922
6,prio_give_way,148,153,809,116,54,0.874595,0.937428,0.904922
7,mand_pass_left_right,22,25,806,115,54,0.875136,0.937209,0.905109
8,prio_stop,66,73,802,114,54,0.875546,0.936916,0.905192
9,mand_right,29,36,801,114,54,0.875410,0.936842,0.905085


In [ ]:
import os
import numpy as np
from collections import defaultdict
import pandas as pd

def load_labels(folder):
    labels = {}
    for filename in os.listdir(folder):
        if filename.endswith('.txt'):
            filepath = os.path.join(folder, filename)
            with open(filepath, 'r') as f:
                lines = f.readlines()
                image_labels = []
                for line in lines:
                    parts = list(map(float, line.strip().split()))
                    image_labels.append(parts)
                labels[filename] = image_labels
    return labels

def iou(box1, box2):
    x1_min = box1[1] - box1[3] / 2
    x1_max = box1[1] + box1[3] / 2
    y1_min = box1[2] - box1[4] / 2
    y1_max = box1[2] + box1[4] / 2
    
    x2_min = box2[1] - box2[3] / 2
    x2_max = box2[1] + box2[3] / 2
    y2_min = box2[2] - box2[4] / 2
    y2_max = box2[2] + box2[4] / 2
    
    inter_x_min = max(x1_min, x2_min)
    inter_x_max = min(x1_max, x2_max)
    inter_y_min = max(y1_min, y2_min)
    inter_y_max = min(y1_max, y2_max)
    
    if inter_x_max < inter_x_min or inter_y_max < inter_y_min:
        return 0.0
    
    intersection = (inter_x_max - inter_x_min) * (inter_y_max - inter_y_min)
    box1_area = (x1_max - x1_min) * (y1_max - y1_min)
    box2_area = (x2_max - x2_min) * (y2_max - y2_min)
    
    union = box1_area + box2_area - intersection
    return intersection / union

def compute_tp_fp_fn(pred_labels, gt_labels, iou_threshold=0.5):
    tp = 0
    fp = 0
    fn = 0
    matched_gt_indices = set()
    
    for pred_box in pred_labels:
        best_iou = 0
        best_gt_index = -1
        for gt_index, gt_box in enumerate(gt_labels):
            if gt_box[0] == pred_box[0]:  # same class
                current_iou = iou(pred_box, gt_box)
                if current_iou > best_iou:
                    best_iou = current_iou
                    best_gt_index = gt_index
        if best_iou > iou_threshold:
            if best_gt_index not in matched_gt_indices:
                tp += 1
                matched_gt_indices.add(best_gt_index)
            else:
                fp += 1
        else:
            fp += 1
    
    fn = len(gt_labels) - len(matched_gt_indices)
    
    return tp, fp, fn

def compute_precision_recall(tp, fp, fn):
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    return precision, recall

def compute_f1(precision, recall):
    if precision + recall == 0:
        return 0
    return 2 * (precision * recall) / (precision + recall)

def compute_metrics(pred_labels, gt_labels, iou_threshold=0.5):
    class_stats = defaultdict(lambda: {'tp': 0, 'fp': 0, 'fn': 0, 'curate_count': 0, 'pred_count': 0})
    
    for image_id in pred_labels:
        preds = pred_labels[image_id]
        gts = gt_labels.get(image_id, [])
        
        tp, fp, fn = compute_tp_fp_fn(preds, gts, iou_threshold)
        
        for pred in preds:
            class_stats[pred[0]]['pred_count'] += 1
        
        for gt in gts:
            class_stats[gt[0]]['curate_count'] += 1  # Curate count is updated here
            
        for class_id, stats in class_stats.items():
            stats['tp'] += tp
            stats['fp'] += fp
            stats['fn'] += fn
    
    results = []
    for class_id, stats in class_stats.items():
        tp, fp, fn = stats['tp'], stats['fp'], stats['fn']
        precision, recall = compute_precision_recall(tp, fp, fn)
        f1 = compute_f1(precision, recall)
        results.append({
            'class_label': int(class_id),
            'Curate count': stats['curate_count'],
            'Pred count': stats['pred_count'],
            'TP': tp,
            'FP': fp,
            'FN': fn,
            'Precision': precision,
            'Recall': recall,
            'F1 Score': f1
        })
    
    return pd.DataFrame(results)

# Define folders
prediction_folder_1 = r'''D:/Projects/Interview/phonologies/assignment/data/Prediction/V1_model_output/labels/'''
prediction_folder_2 = r'''D:/Projects/Interview/phonologies/assignment/data/Prediction/V2_model_output/labels/'''
ground_truth_folder = r'''D:/Projects/Interview/phonologies/assignment/data/YOLO_Data/val/labels/'''

# Load labels
pred_labels_1 = load_labels(prediction_folder_1)
pred_labels_2 = load_labels(prediction_folder_2)
gt_labels = load_labels(ground_truth_folder)

# Check ground truth consistency
print("Ground Truth Labels:")
for img_id, labels in gt_labels.items():
    print(f"{img_id}: {len(labels)} labels")

# # Compute metrics for first prediction set
# class_stats_df_1 = compute_metrics(pred_labels_1, gt_labels)
# print("\nMetrics for first prediction set:")
# print(class_stats_df_1)

# # Compute metrics for second prediction set
# class_stats_df_2 = compute_metrics(pred_labels_2, gt_labels)
# print("\nMetrics for second prediction set:")
# print(class_stats_df_2)


In [15]:
def count_gt_labels_per_class(gt_labels):
    class_counts = defaultdict(int)
    
    for image_id, labels in gt_labels.items():
        for label in labels:
            class_id = int(label[0])  # Assuming the first element is the class ID
            class_counts[class_id] += 1
    
    return class_counts

prediction_folder_1 = r'''D:/Projects/Interview/phonologies/assignment/data/Prediction/V1_model_output/labels/'''
prediction_folder_2 = r'''D:/Projects/Interview/phonologies/assignment/data/Prediction/V2_model_output/labels/'''
ground_truth_folder = r'''D:/Projects/Interview/phonologies/assignment/data/YOLO_Data/val/labels/'''

# Load ground truth labels
gt_labels = load_labels(prediction_folder_1)

# Count ground truth labels per class
class_counts = count_gt_labels_per_class(gt_labels)

# # Print the counts
# print("Ground Truth Labels Count per Class:")
# for class_id, count in class_counts.items():
#     print(f"Class {class_id}: {count} labels")


In [16]:
type(class_counts)

collections.defaultdict

In [17]:
# Extract data for columns and rows
col_names = list(class_counts.keys())
col_values = list(class_counts.values())

# Create DataFrame
df = pd.DataFrame(list(zip(col_values)), columns=col_names)
print(df)

ValueError: 45 columns passed, passed data had 1 columns

In [18]:
len(col_names)

45

In [19]:
len(col_values)

45

In [7]:
cls_gt = pd.DataFrame(class_counts, index=[0])
cls_gt = cls_gt.T

In [8]:
cls_gt.shape

(40, 1)

In [47]:
pd.DataFrame(dict(class_counts))

ValueError: If using all scalar values, you must pass an index

In [41]:
# Load ground truth labels
gt_labels = load_labels(ground_truth_folder)

# Check ground truth consistency
print("Ground Truth Labels Count:")
total_gt_labels = 0
for img_id, labels in gt_labels.items():
    print(f"{img_id}: {len(labels)} labels")
    total_gt_labels += len(labels)
print(f"Total ground truth labels: {total_gt_labels}")


Ground Truth Labels Count:
SNAG-00013_png_jpg.rf.5655f1d044e0f5cccc6cf3c17757e6e0.txt: 1 labels
SNAG-00014_png_jpg.rf.69143aca9eb36598083bb8c9ff5e97f3.txt: 1 labels
SNAG-00015_png_jpg.rf.62a9e36011a59c52ce76c3067f3d6f53.txt: 1 labels
SNAG-00016_png_jpg.rf.a20b6a9a3f604f1d619e18b7a507b4d0.txt: 1 labels
SNAG-00017_png_jpg.rf.a4ece30ed6fd85ecc65ad4e8cc8f195b.txt: 1 labels
SNAG-00018_png_jpg.rf.baa809519cc98eabcc554485dc2fffeb.txt: 2 labels
SNAG-00019_png_jpg.rf.85ab5f5947d5a4bef9c1fd2c9a281592.txt: 1 labels
SNAG-00020_png_jpg.rf.23dcdf354340d07c6bccf60aa66df086.txt: 1 labels
SNAG-00021_png_jpg.rf.68eecfd6d34c86098af753ac85d1fe80.txt: 3 labels
SNAG-00023_png_jpg.rf.c6083c92ec4e12b15c0c2005c32cdf32.txt: 3 labels
SNAG-00024_png_jpg.rf.4c3f15ca5c7e8d4447d6d06861174fb4.txt: 1 labels
SNAG-00025_png_jpg.rf.9ba1a5d3ea6f7665b968ed27d7ff911d.txt: 1 labels
SNAG-00026_png_jpg.rf.2fefadf5d73d03635b1588798b6f0b11.txt: 3 labels
SNAG-00027_png_jpg.rf.f3e99d49df2c2c21b2dca8c6b091a3bc.txt: 3 labels
SNAG-00